## **RADI608: Data Mining and Machine Learning**

### Assignment: Genetic Algorithm
**Romen Samuel Rodis Wabina** <br>
Student, PhD Data Science in Healthcare and Clinical Informatics <br>
Clinical Epidemiology and Biostatistics, Faculty of Medicine (Ramathibodi Hospital) <br>
Mahidol University

##### Suppose $w_i$ is a function weight where $w$ is an integer number between <code>range[0, 10)</code> such that $i = [1, 2, 3, 4, 5, 6]$. We first initialized a population that consists of eight (8) chromosomes with 6 genes per chromosome. This is represented in the figure below. I used <code>random.randint()</code> to help me draw random numbers from the given range with shape <code>(8, 6)</code>.

<center>
<img src = "../figures/11.jpg" width = '350'> <br>
</center>

Suppose $\mathbf{y_i}$ is the fitness score of each chromosome $i \in [1, 6]$. We evaluate the fitness of each chromosome to the given function $\mathbf{y} = w_1x_1^2 + w_2x_2^3 + w_3x_3 + w_4x_4 + w_5x_5 + w_6x_6$ with given inputs $(x_1, x_2, x_3, x_4, x_5, x_6)$ = <code>(4, -2, 7, -5, 11, 1)</code>

\begin{equation*}
\begin{aligned}
    \mathbf{y_1} &= w_1x_1^2 + w_2x_2^3 + w_3x_3 + w_4x_4 + w_5x_5 + w_6x_6 \\
    \mathbf{y_1} &= (0)(4)^2 + (6)(-2)^3 + (4)(7) + (0)(-5) + (4)(11) + (5)(1) \\
    \mathbf{y_1} &= 29
\end{aligned}
\end{equation*}

Doing the same calculation to other $\mathbf{y}$, the fitness scores from chromosome 1 to 6 are <code>(29, 7, 240, 22, 61, 72, 60, 114)</code>. 

**Generation 1: PARENT SELECTION** Parents are used as the basis for generating the next generation of candidate points and one parent for each position in the population is required. Parents are then taken in pairs and used to create two children. Recombination is performed using a crossover operator. This involves selecting a random split point on the bit string, then creating a child with the bits up to the split point from the first parent and from the split point to the end of the string from the second parent. This process is then inverted for the second child. 

To do this, we selected the parents (i.e., chromosomes) for the next generation using a roulette wheel. We partitioned the ratio of fitness scores from each chromosome by getting the relative proportion (i.e., the probability for selecting the chromotion proportionate to its fitness value). Hence, its relative portion are <code>[0.0479, 0.0116, 0.3967, 0.0364, 0.1008, 0.119, 0.0992, 0.1884]</code>. The relative portion is best represented in the following diagram. Each time the wheel is turned, the selection point is used to choose a single individual from the entire population. The wheel is then turned again to select the next individual until we have enough individuals selected to fill the next generation. 

We have four pairs of parents since there are eight chromosomes in this problem. Using <code>random.uniform(low = 0.0, high = 1.0)</code>, we randomly choose eight values to determine the parents (i.e., Daddy and Mommy) of the next generation. Therefore,

    Pair 1
        - Random 1: 0.00164, then choose chromosome 1 as Daddy
        - Random 2: 0.05346, then choose chromosome 2 as Mommy

    Pair 2
        - Random 3: 0.4428, then choose chromosome 3 as Daddy
        - Random 4: 0.4792. then choose chromosome 4 as Mommy

    Pair 3
        - Random 5: 0.5512, then choose chromosome 5 as Daddy
        - Random 6: 0.6232, then choose chromosome 6 as Mommy

    Pair 4
        - Random 7: 0.7623, then choose chromosome 7 as Daddy
        - Random 8: 0.9913, then choose chromosome 8 as Mommy


<center>
<img src = "../figures/12.jpg" width = '1300'> <br>
</center>

**Generation 1: MUTATION** We then randomly selected six mutation probabilities for every chromosome to examine whether the offspring chromosome will mutate or not. The gene will mutate if its probability is less than the given mutation rate 0.1. From the figure below, we have shown that arrays in colored red are mutated from their original values. Let's say look at chromosome (a), since both genes 0 have mutation probabilities (0.0113 and 0.0626) less than 0.1, then we mutated it into other values. We used the <code>random.randint()</code> to help us draw random integer values between <code>[0, 10)</code>. Only the chromosome (b) and (h) did not mutate its values.

<center>
<img src = "../figures/13.jpg" width = '900'> <br>
</center>

We evaluated the fitness score to all mutated offsprings using the given fitness function $\mathbf{y}$. We then sorted these chromosomes from highest fitness score to lowest and get the top 8 chromosomes. These eight chromosomes will become the new population for the generation 2. The fitness scores of the chromosomes listed below are <code>240, 1810, 119, 115, 114, 73, 72, 611</code>, respectively.

<center>
<img src = "../figures/112.jpg" width = 300'> <br>
</center>

**Generation 2: PARENT SELECTION** Similarly, we then now select the parents (i.e., chromosomes) for the **generation 2** using a roulette wheel. We partitioned the ratio of fitness scores from each chromosome by getting the relative proportion (i.e., the probability for selecting the chromotion proportionate to its fitness value). Hence, its relative portion are <code>[0.0479, 0.0116, 0.3967, 0.0364, 0.1008, 0.119, 0.0992, 0.1884]</code>. The relative portion is best represented in the following diagram. Each time the wheel is turned, the selection point is used to choose a single individual from the entire population. The wheel is then turned again to select the next individual until we have enough individuals selected to fill the next generation. 

We have four pairs of parents since there are eight chromosomes in this problem. Using <code>random.uniform(low = 0.0, high = 1.0)</code>, we randomly choose eight values to determine the parents (i.e., Daddy and Mommy) of the next generation. Therefore,

    Pair 1
        - Random 1: 0.6108, then choose chromosome 4 as Daddy
        - Random 2: 0.9006, then choose chromosome 7 as Mommy

    Pair 2
        - Random 3: 0.0501, then choose chromosome 1 as Daddy
        - Random 4: 0.6380, then choose chromosome 4 as Mommy

    Pair 3
        - Random 5: 0.3523, then choose chromosome 2 as Daddy
        - Random 6: 0.6742, then choose chromosome 5 as Mommy

    Pair 4
        - Random 7: 0.1979, then choose chromosome 1 as Daddy
        - Random 8: 0.8934, then choose chromosome 7 as Mommy

<center>
<img src = "../figures/14.jpg" width = '1300'> <br>
</center>

**Generation 2: MUTATION** We randomly selected anothersix mutation probabilities for every chromosome to examine whether the offspring chromosome will mutate or not. The gene will mutate if its probability is less than the given mutation rate 0.1. From the figure below, we have shown that arrays in colored red are mutated from their original values. Let's say look at chromosome (a), since both genes 0 have mutation probabilities (0.0113 and 0.0626) less than 0.1, then we mutated it into other values. We used the <code>random.randint()</code> to help us draw random integer values between <code>[0, 10)</code>. Only the chromosome (b) and (h) did not mutate its values.

<center>
<img src = "../figures/15.jpg" width = '900'> <br>
</center>

**Generation 2: EVALUATION OF FITNESS FUNCTION** We finally sort the parent chromosomes and child chromosomes based on pool to population. The figure below shows the top 8 chromosomes, along with their respective fitness score.
<center>
<img src = "../figures/16.jpg" width = '500'> <br>
</center>

In [106]:
import sys
import numpy as np
import GA as ga
import random 
import matplotlib.pyplot as plt

equation_inputs = [4, -2, 7, -5, 11, 1]
num_weights = len(equation_inputs)
chromosomes = 8
mating = 4

random.seed(413)
new_population = np.random.uniform(low = 0, high = 10, size = (chromosomes, num_weights))
initial_population = [  [0, 6, 4, 0, 4, 5],
                        [5, 7, 0, 5, 0, 8],
                        [9, 0, 7, 1, 4, 8],
                        [2, 4, 0, 1, 2, 5],
                        [9, 9, 0, 7, 2, 2],
                        [3, 5, 7, 7, 4, 6],
                        [6, 7, 4, 5, 1, 6],
                        [7, 8, 3, 7, 7, 3]]
                    

In [68]:
def cal_fitness(inputs, initial_population):
    ''' 
    We changed the cal_pop_fitness into cal_fitness
    This follows the equation: 
        y = w1 * (x1 ** 2) + w2 * (x2 ** 3) + (w3 * x3) + (w4 * x4) + (w5 * x5) + (w6 * x6)
    The equation has six inputs and six weights
    '''
    fitness = inputs[0] * (initial_population[:, 0] ** 2)   + \
              inputs[1] * (initial_population[:, 1] ** 3)   + \
              inputs[2] * (initial_population[:, 2])        + \
              inputs[3] * (initial_population[:, 3])        + \
              inputs[4] * (initial_population[:, 4])        + \
              inputs[5] * (initial_population[:, 5])
    return fitness

def crossover(parents, offspring_size, crossover_rate = 0.1):
    offspring = np.empty(offspring_size)
    crossover_point = np.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        parent1_idx = k%parents.shape[0]
        parent2_idx = (k+1)%parents.shape[0]

        random.seed(413)
        crossover_prob = np.random.uniform(0.0, 0.8, 1)

        if crossover_prob < crossover_rate:
            offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
            offspring[k, crossover_point: ] = parents[parent2_idx, crossover_point: ]
    return offspring

def mutation(offspring_crossover, mutations_rate):
    mutations_counter = np.uint8(offspring_crossover.shape[1])
    for idx in range(offspring_crossover.shape[0]):
        gene_idx = mutations_counter - 1

        random.seed(413)
        random_value  = np.random.uniform(-1.0, 1.0, 1)
        mutation_prob = np.random.uniform(-1.0, 1.0, 1)

        if mutation_prob < mutations_rate:
            offspring_crossover[idx, gene_idx] = offspring_crossover[idx, gene_idx] + random_value
            gene_idx = gene_idx + mutations_counter
    return offspring_crossover

def main_genetic(equation_inputs, new_population, chromosomes, mating):
    num_weights = len(equation_inputs)
    sol_per_pop = chromosomes
    num_parents_mating = mating
    pop_size = (sol_per_pop, num_parents_mating)

    best_outputs = []
    num_generations = 2

    crossover_rate = 0.8 
    mutations_rate = 0.1

    for generation in range(num_generations):
        fitness = cal_fitness(equation_inputs, new_population)
        best_outputs.append(np.max(fitness))

        parents = ga.select_mating_pool(new_population, fitness, num_parents_mating)

        offspring_size = (pop_size[0] - parents.shape[0], num_weights)
        offspring_crossover = crossover(parents, 
                                        offspring_size = offspring_size, 
                                        crossover_rate = crossover_rate)

        offspring_mutation  = mutation(offspring_crossover, 
                                       mutations_rate = mutations_rate)

        new_population[0:parents.shape[0], :] = parents
        new_population[parents.shape[0]:,  :] = offspring_mutation

    fitness = cal_fitness(equation_inputs, new_population)
    best_match_idx = np.where(fitness == np.max(fitness))
    print('Best approximate weight solution:  \t', np.round(new_population[best_match_idx, :], 2)[0])
    print('Best approximate solution fitness: \t', np.round(fitness[best_match_idx][0], 3))
    return best_outputs

best_outputs = main_genetic(equation_inputs, new_population, chromosomes, mating)

Best approximate weight solution:  	 [[7.   2.62 9.78 4.19 8.81 3.62]]
Best approximate solution fitness: 	 308.39


In [97]:
from numpy.random import randint
from numpy.random import rand

initial_population = [randint(0, 10, 6).tolist() for _ in range(8)]
initial_population = [  [0, 6, 4, 0, 4, 5],
                        [5, 7, 0, 5, 0, 8],
                        [9, 0, 7, 1, 4, 8],
                        [2, 4, 0, 1, 2, 5],
                        [9, 9, 0, 7, 2, 2],
                        [3, 5, 7, 7, 4, 6],
                        [6, 7, 4, 5, 1, 6],
                        [7, 8, 3, 7, 7, 3]]

mutated_population = [  [3, 6, 4, 5, 2, 8],
                        [5, 7, 0, 0, 4, 5],
                        [9, 0, 7, 8, 2, 5],
                        [3, 4, 0, 3, 4, 4],
                        [9, 5, 0, 7, 4, 2],
                        [3, 5, 2, 3, 2, 2],
                        [6, 7, 4, 7, 7, 9],
                        [7, 8, 3, 5, 1, 6]]

mutated_population2 = [ [9, 1, 0, 7, 4, 6],
                        [5, 1, 2, 7, 4, 2],
                        [9, 0, 7, 7, 4, 3],
                        [9, 5, 0, 1, 4, 5],
                        [9, 0, 7, 7, 4, 3],
                        [9, 5, 3, 8, 2, 5]]


# sorted_population = [ 
#     [9, 0, 7, 1, 4, 8] (240), 
#     [9, 0, 7, 8, 2, 5] (180),
#     [6, 7, 4, 7, 7, 9] (119),
#     [9, 5, 0, 7, 4, 2] (115),
#     [7, 8, 3, 7, 7, 3] (114),
#     [5, 7, 0, 0, 4, 5]  (73),
#     [3, 5, 7, 7, 4, 6]  (72),
#     [9, 9, 0, 7, 2, 2]  (61),
#     [7, 8, 3, 5, 1, 6]  (61),
#     [6, 7, 4, 5, 1, 6]  (60),
#     [3, 4, 0, 3, 4, 4]  (49),
#     [3, 6, 4, 5, 2, 8]  (33),
#     [3, 5, 2, 3, 2, 2]  (31),
#     [0, 6, 4, 0, 4, 5]  (29),
#     [2, 4, 0, 1, 2, 5]  (22),
#     [5, 7, 0, 5, 0, 8]   (7)
# ]

new_population = [
    [9, 0, 7, 1, 4, 8], 
    [9, 0, 7, 8, 2, 5],
    [6, 7, 4, 7, 7, 9],
    [9, 5, 0, 7, 4, 2],
    [7, 8, 3, 7, 7, 3],
    [5, 7, 0, 0, 4, 5],
    [3, 5, 7, 7, 4, 6],
    [9, 9, 0, 7, 2, 2]
]

In [102]:
def scorer(w, equation_inputs):
    return (w[0] * (equation_inputs[0] ** 2)) + \
           (w[1] * (equation_inputs[1] ** 3)) + \
           (w[2] * equation_inputs[2]) + \
           (w[3] * equation_inputs[3]) + \
           (w[4] * equation_inputs[4]) + \
           (w[5] * equation_inputs[5])

equation_inputs = [4, -2, 7, -5, 11, 1]
decoded = [1, 5, 30, 7, 16, 15, 46, 4]

scores1 = []
for decode in initial_population:
    scores1.append(scorer(decode, equation_inputs))
scores1

[29, 7, 240, 22, 61, 72, 60, 114]

In [101]:
scores2 = []
for decode in mutated_population:
    scores2.append(scorer(decode, equation_inputs))


scores4 = []
for decode in mutated_population2:
    scores4.append(scorer(decode, equation_inputs))
print(scores4)

scores3 = []
for decode in new_population:
    scores3.append(scorer(decode, equation_inputs))

[151, 97, 205, 148, 205, 112]


In [103]:
for_selection = scores1.copy()
selection_by_parents = []
for idx in for_selection:
    sum_value = sum(for_selection)
    selection_by_parents.append(np.round(idx/sum_value, 4))
selection_by_parents

[0.0479, 0.0116, 0.3967, 0.0364, 0.1008, 0.119, 0.0992, 0.1884]

In [104]:
selection = []
for select in range(len(selection_by_parents)):
    selection.append(sum(selection_by_parents[0:select]))
selection.append(1)
selection

[0,
 0.0479,
 0.0595,
 0.4562,
 0.4926,
 0.5933999999999999,
 0.7123999999999999,
 0.8115999999999999,
 1]

In [90]:
np.random.uniform(low = 0.0, high = 1.0)

0.4783300074438507

In [91]:
np.round(np.random.uniform(low = 0.0, high = 1.0, size = (chromosomes, num_weights)), 4)

array([[0.6828, 0.0062, 0.2515, 0.3428, 0.8188, 0.2273],
       [0.0816, 0.3605, 0.0124, 0.2567, 0.682 , 0.5007],
       [0.0997, 0.2224, 0.5151, 0.0321, 0.0412, 0.2057],
       [0.3686, 0.807 , 0.8277, 0.9478, 0.7085, 0.896 ],
       [0.6697, 0.813 , 0.8659, 0.91  , 0.4447, 0.2236],
       [0.8549, 0.134 , 0.2398, 0.4762, 0.2396, 0.2434],
       [0.976 , 0.7453, 0.4887, 0.2291, 0.763 , 0.0746],
       [0.1923, 0.6066, 0.3283, 0.1795, 0.2903, 0.7262]])

In [96]:
np.random.randint(low = 0, high = 10)

3

In [26]:
def objective(x):
	return x[0]**2.0 + x[1]**.0
 
# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
	decoded = list()
	largest = 2**n_bits
	for i in range(len(bounds)):
		# extract the substring
		start, end = i * n_bits, (i * n_bits)+n_bits
		substring = bitstring[start:end]
		# convert bitstring to a string of chars
		chars = ''.join([str(s) for s in substring])
		# convert string to integer
		integer = int(chars, 2)
		# scale integer to desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
		# store
		decoded.append(value)
	return decoded
 
def selection(pop, scores, k = 3):
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]
 
def crossover(p1, p2, r_cross):
	c1, c2 = p1.copy(), p2.copy()
	if rand() < r_cross:
		pt = randint(1, len(p1)-2)
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]
 
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		if rand() < r_mut:
			bitstring[i] = 1 - bitstring[i]
 
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
	# pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
	# # pop = [randint(0, 2, 6).tolist() for _ in range(8)]
	pop =  [[0, 0, 0, 0, 0, 1],
			[0, 0, 0, 1, 0, 1],
			[0, 1, 1, 1, 1, 0],
			[0, 0, 0, 1, 1, 1],
			[0, 1, 0, 0, 0, 0],
			[0, 0, 1, 1, 1, 1],
			[1, 0, 1, 1, 1, 1],
			[0, 0, 0, 1, 0, 0]]
	print(decode(bounds, n_bits, pop[0]))
	best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
	for gen in range(n_iter):
		decoded = [decode(bounds, n_bits, p) for p in pop]
		scores = [objective(d) for d in decoded]

		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print("Iteration \t %d, New best f(%s) = %f" % (gen,  decoded[i], scores[i]))

		selected = [selection(pop, scores) for _ in range(n_pop)]

		children = list()
		for i in range(0, n_pop, 2):
			p1, p2 = selected[i], selected[i+1]
			for c in crossover(p1, p2, r_cross):
				mutation(c, r_mut)
				children.append(c)
		pop = children
	return [best, best_eval]
 
# # define range for input
# bounds = [[-5.0, 5.0], [-5.0, 5.0]]
# # define the total iterations
# n_iter = 100
# # bits per variable
# n_bits = 6
# n_pop = 8
# r_cross = 0.8
# r_mut = 0.1
# best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
# print('Done!')
# decoded = decode(bounds, n_bits, best)
# print('f(%s) = %f' % (decoded, score))

In [15]:
initial_population = [randint(0, 10, 6).tolist() for _ in range(8)]
initial_population

[[5, 0, 3, 0, 0, 8],
 [7, 7, 2, 0, 7, 1],
 [8, 5, 2, 1, 7, 9],
 [0, 8, 4, 0, 7, 9],
 [4, 4, 3, 0, 3, 3],
 [6, 7, 0, 6, 5, 7],
 [0, 2, 9, 3, 5, 8],
 [8, 6, 0, 5, 8, 3]]

In [4]:
pop =  [[0, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 1],
        [0, 1, 1, 1, 1, 0],
        [0, 0, 0, 1, 1, 1],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 1],
        [1, 0, 1, 1, 1, 1],
        [0, 0, 0, 1, 0, 0]]

pop

[[0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 1],
 [0, 1, 1, 1, 1, 0],
 [0, 0, 0, 1, 1, 1],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 1, 1, 1, 1],
 [1, 0, 1, 1, 1, 1],
 [0, 0, 0, 1, 0, 0]]